## Preparations

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math
import numpy as np
from scipy.spatial import distance


USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")
#device = torch.device("cpu")

## Load & Preprocess Data 

Let's have a look to our dialydialog dataset : https://www.aclweb.org/anthology/I17-1099/

In [2]:
corpus_name = "dailydialog"
corpus = os.path.join("data", corpus_name)

def printLines(file, n=10):
    with open(file, 'r', encoding="utf-8") as datafile:
        lines = datafile.readlines()
    for line in lines[:n]:
        print(line)

printLines(os.path.join(corpus, "dialogues_text.txt"))

The kitchen stinks . __eou__ I'll throw out the garbage . __eou__

So Dick , how about getting some coffee for tonight ? __eou__ Coffee ? I don ’ t honestly like that kind of stuff . __eou__ Come on , you can at least try a little , besides your cigarette . __eou__ What ’ s wrong with that ? Cigarette is the thing I go crazy for . __eou__ Not for me , Dick . __eou__

Are things still going badly with your houseguest ? __eou__ Getting worse . Now he ’ s eating me out of house and home . I ’ Ve tried talking to him but it all goes in one ear and out the other . He makes himself at home , which is fine . But what really gets me is that yesterday he walked into the living room in the raw and I had company over ! That was the last straw . __eou__ Leo , I really think you ’ re beating around the bush with this guy . I know he used to be your best friend in college , but I really think it ’ s time to lay down the law . __eou__ You ’ re right . Everything is probably going to come to a head to

## Create formatted data file

We'll create a formatted data file in which each line contains a tab-separated query sentence and a response sentence pair.

The following functions pase the `dailogues_text.txt` data file.
- `loadLines` splits each line of the file into conversations
- `extractSentencePairs` extracts pairs of sentences from conversations

In [8]:
# Splits each line of the file into a dictionary of fields
def loadLines(fileName):
    conversations = []
    with open(fileName, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split("__eou__")
            conversations.append(values)
    return conversations


# Extracts pairs of sentences from conversations
def extractSentencePairs(conversations):
    qa_pairs = []
    for conversation in conversations:
        # Iterate over all the lines of the conversation
        for i in range(len(conversation) - 1):  # We ignore the last line (no answer for it)
            inputLine = conversation[i].strip()
            targetLine = conversation[i+1].strip()
            # Filter wrong samples (if one of the lists is empty)
            if inputLine and targetLine:
                qa_pairs.append([inputLine, targetLine])
    return qa_pairs

Now we call the above functions to create a new file : `formatted_dialogues_text.txt`

In [9]:
# Define path to new file
datafile = os.path.join(corpus, "formatted_dialogues_text.txt")

delimiter = '\t'
# Unescape the delimiter
delimiter = str(codecs.decode(delimiter, "unicode_escape"))

print("\nLoading conversations...")
conversations = loadLines(os.path.join(corpus, "dialogues_text.txt"))

# Write new csv file
print("\nWriting newly formatted file...")
with open(datafile, 'w', encoding='utf-8') as outputfile:
    writer = csv.writer(outputfile, delimiter=delimiter, lineterminator='\n')
    for pair in extractSentencePairs(conversations):
        writer.writerow(pair)

# Print a sample of lines
print("\nSample lines from file:")
printLines(datafile)


Loading conversations...

Writing newly formatted file...

Sample lines from file:
The kitchen stinks .	I'll throw out the garbage .

So Dick , how about getting some coffee for tonight ?	Coffee ? I don ’ t honestly like that kind of stuff .

Coffee ? I don ’ t honestly like that kind of stuff .	Come on , you can at least try a little , besides your cigarette .

Come on , you can at least try a little , besides your cigarette .	What ’ s wrong with that ? Cigarette is the thing I go crazy for .

What ’ s wrong with that ? Cigarette is the thing I go crazy for .	Not for me , Dick .

Are things still going badly with your houseguest ?	Getting worse . Now he ’ s eating me out of house and home . I ’ Ve tried talking to him but it all goes in one ear and out the other . He makes himself at home , which is fine . But what really gets me is that yesterday he walked into the living room in the raw and I had company over ! That was the last straw .

Getting worse . Now he ’ s eating me out of 

## Load and trim data

Now let's create a vocabulary and load query/response sentence pairs into memory.

First we must create a mapping of each word to a discrete numerical space (the index value).

Voc class keeps the mapping from words to indexes, a reverse mapping of indexes to words, a count of each word and a total word count.
There are 3 central methods :
- `addWord` to add a word to the vocabulary
- `addSentence` to add all words in a sentence
- `trim` for trimming infrequently seen words

In [10]:
# Default word tokens
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3  # Count SOS, EOS, PAD

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    # Remove words below a certain count threshold
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        # Reinitialize dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # Count default tokens

        for word in keep_words:
            self.addWord(word)

Before assemble our vocabulary and query/response sentence pairs we must perform some preprocessing.

1. Convert the Unicode strings to ASCII with `unicodeToAscii`.
2. Convert all letters to lowercase and trim all non-letter characters except basic punctuation `normalizeString`
3. Filter out sentences witg length greater than the `MAX_LENGTH` threshold in `filterPairs`


In [11]:
#handle dull_responses now
dull_responses = ["I do not know what you are talking about.", "I do not know.", 
 "You do not know.", "You know what I mean.", "I know what you mean.", 
 "You know what I am saying.", "You do not know anything."]

In [12]:
MAX_LENGTH = 15  # Maximum sentence length to consider

# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

# Read query/response pairs and return a voc object
def readVocs(datafile, corpus_name):
    print("Reading lines...")
    # Read the file and split into lines
    lines = open(datafile, encoding='utf-8').\
        read().strip().split('\n')
    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    voc = Voc(corpus_name)
    return voc, pairs

# Returns True iff both sentences in a pair 'p' are under the MAX_LENGTH threshold
def filterPair(p):
    # Input sequences need to preserve the last word for EOS token
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

# Filter pairs using filterPair condition
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]


# Using the functions defined above, return a populated voc object and pairs list
def loadPrepareData(corpus, corpus_name, datafile, save_dir):
    print("Start preparing training data ...")
    voc, pairs = readVocs(datafile, corpus_name)
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words...")
    for d in dull_responses:
        voc.addSentence(d)
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words:", voc.num_words)
    return voc, pairs

Finally assmble voc and pairs

In [13]:
# Load/Assemble voc and pairs
save_dir = os.path.join("data", "save")
voc, pairs = loadPrepareData(corpus, corpus_name, datafile, save_dir)
# Print some pairs to validate
print("\npairs:")
for pair in pairs[:10]:
    print(pair)

Start preparing training data ...
Reading lines...
Read 89862 sentence pairs
Trimmed to 43724 sentence pairs
Counting words...
Counted words: 10126

pairs:
['the kitchen stinks .', 'i ll throw out the garbage .']
['so dick how about getting some coffee for tonight ?', 'coffee ? i don t honestly like that kind of stuff .']
['coffee ? i don t honestly like that kind of stuff .', 'come on you can at least try a little besides your cigarette .']
['would you mind waiting a while ?', 'well how long will it be ?']
['i swear i m going to kill you for this .', 'what s wrong ? didn t you think it was fun ? !']
['never ! but thank you for inviting me .', 'come on . you ll feel better after we hit the showers .']
['certainly . how about spaghetti with clams and shrimps .', 'sounds delicious . ok . she ll try that .']
['can you manage chopsticks ?', 'why not ? see .']
['why not ? see .', 'good mastery . how do you like our chinese food ?']
['i m exhausted .', 'okay let s go home .']


### Trimming rarely used words out of vocab

One tactic beneficial to achieve faster convergence during training is trimming rarely used words out of our vocabulary.

1. Trim words used under `MIN_COUNT` threshold using `voc.trim`
2. Filter out pairs with trimmed words

In [14]:
MIN_COUNT = 3    # Minimum word count threshold for trimming

def trimRareWords(voc, pairs, MIN_COUNT):
    # Trim words used under the MIN_COUNT from the voc
    voc.trim(MIN_COUNT)
    # Filter out pairs with trimmed words
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input = True
        keep_output = True
        # Check input sentence
        for word in input_sentence.split(' '):
            if word not in voc.word2index:
                keep_input = False
                break
        # Check output sentence
        for word in output_sentence.split(' '):
            if word not in voc.word2index:
                keep_output = False
                break

        # Only keep pairs that do not contain trimmed word(s) in their input or output sentence
        if keep_input and keep_output:
            keep_pairs.append(pair)

    print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)))
    return keep_pairs


# Trim voc and pairs
pairs = trimRareWords(voc, pairs, MIN_COUNT)

keep_words 6159 / 10123 = 0.6084
Trimmed from 43724 pairs to 38716, 0.8855 of total


## Prepare Data for Models

BATCH TECHNIQUE

To accomodate sentences of different sizes in the same batch we make our batched input tensor of shape `(max_length, batch_size)` where sentences shorter than the max_length are zeropadded after the `EOS_token`.

- `inputVar` function handles the process of converting sentences to tensor. It returns a tensor of `lengths` for each sequence in the batch for the decoder.
- `outputVar` function performs the same as `inputVar` but returns a binary mask tensor and a maximum target sentence length.
- `batch2TrainData` takes a bunch of pairs and returns the input and target tensors

In [15]:
def indexesFromSentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]


def zeroPadding(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# Returns padded input sequence tensor and lengths
def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

# Returns padded target sequence tensor, padding mask, and max target length
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.BoolTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

# Returns all items for a given batch of pairs
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len


# Example for validation
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("input_variable:", input_variable)
print("lengths:", lengths)
print("target_variable:", target_variable)
print("mask:", mask)
print("max_target_len:", max_target_len)

input_variable: tensor([[  16,   16,   80,  159,  109],
        [  34,   77,  167,   64,  143],
        [  62, 1902,  304,  357,   15],
        [ 574,   28,  288,  937,    2],
        [  19,   13, 5314,   28,    0],
        [  84,  676,  409,    8,    0],
        [ 146,  355,  766,   30,    0],
        [ 572,   15,   15,    2,    0],
        [ 576,    2,    2,    0,    0],
        [ 774,    0,    0,    0,    0],
        [  86,    0,    0,    0,    0],
        [ 102,    0,    0,    0,    0],
        [ 771,    0,    0,    0,    0],
        [  15,    0,    0,    0,    0],
        [   2,    0,    0,    0,    0]])
lengths: tensor([15,  9,  9,  8,  4])
target_variable: tensor([[  50,   16,   94,  109,   23],
        [   8,   77,   30,   57,  374],
        [  34,   13,    2,  152,   28],
        [  62, 1365,    0,  205,   13],
        [ 102,  487,    0,  676, 1362],
        [  19,   15,    0, 1514,   30],
        [  26,    2,    0,   15,    2],
        [ 432,    0,    0,    2,    0],
        

## Seq2Seq

The brain of our chatbot is a sequence to sequence model.

One RNN acts as an _encoder_  which encodes a variable length input sequence to a fixed-length context vector (the final hidden layer of the RNN).
The second RNN is a _decoder_ which takes a s input a word and a context vector and returns a guess for the next word in the sequence.

 ### Encoder

The encoder RNN iterates through tokens and outputs an "output" vector and a "hidden sate" vector. the hidden state vector is passed to the next time step while the output vector is recorder.
The encoder transforms the context it saw at each point in the sequence into a set of points in a high dimensional space. The decoder will use it to generate the outputted word.

We use a bidirectional multi-layered Gated Recurrent Unit.
It gives the advantage of encoding both past and future context !

Computation Graph :
1. Convert word indexed to embeddings
2. Pack padded batch of sequences for RNN module
3. Forward pass through GRU
4. Unpack padding
5. Sum bidirectional GRU outputs
6. Return output and final hidden state

In [16]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        # Initialize GRU; the input_size and hidden_size params are both set to 'hidden_size'
        #   because our input size is a word embedding with number of features == hidden_size
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths, hidden=None):
        # Convert word indexes to embeddings
        embedded = self.embedding(input_seq)
        # Pack padded batch of sequences for RNN module
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        # Forward pass through GRU
        outputs, hidden = self.gru(packed, hidden)
        # Unpack padding
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        # Sum bidirectional GRU outputs
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # Return output and final hidden state
        return outputs, hidden

### Decoder

The decoder RNN uses the encoder's context vectors and internal hidden states to generate the next word of the sequence.
It continues generating words until an `EOS_token`.
The problem with a vanilla seq2seq decoder is that if we rely woley on the context vector it will have information loss. (especially with long input sequences).

-> `attention mechanism` allows the decoder to pay attention to certain parts of the input sequence.

In [17]:
# Luong attention layer
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights (energies) based on the given method
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        # Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.t()

        # Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

Now that the attention submodule is implemented let's dive into the actual decoder model.

Computation Graph:
1. Get embedding of current input word.
2. Forward through unidirectional GRU
3. Calculate attention weights from the current GRU output
4. Multiply attention weights to encoder outputs to get a new context vector
5. Concatenate weighted context vector and GRU using Luong eq
6. Predict next word
7. Return output and final hidden state

In [18]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # Define layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        # Note: we run this one step (word) at a time
        # Get embedding of current input word
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        # Forward through unidirectional GRU
        rnn_output, hidden = self.gru(embedded, last_hidden)
        # Calculate attention weights from the current GRU output
        attn_weights = self.attn(rnn_output, encoder_outputs)
        # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        # Concatenate weighted context vector and GRU output using Luong eq. 5
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        # Predict next word using Luong eq. 6
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        # Return output and final hidden state
        return output, hidden

## Training 

`maskNNLLLoss` calculates the average negative log likelihood of the elements that correspond to a 1 in the mask tensor.

In [19]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

Single training iteration (single batch of inputs)

Couple of clever tricks :
- `teacher forcing` at some probability (set by `teacher_forcing_ratio`) current target word is used as the decoder's next input rather than using the decoder's current guess.
- `gradient clipping` commonly used technique for countering the "exploding gradient" problem.

Sequence of operations :
1. Forward pass entire input batch through encoder
2. initialize decoder inputs as SOS_token and hidden state as the encoder's final hidden state
3. Forward input batch sequence through decoder one time step at a time
4. If teacher forcing : set next decoder input as the current target else : set next decoder input as the current decoder ouptput
5. Calculate and accumulate loss
6. Perform backpropagation
7. Clip gradients
8. Update encoder and decoder

In [20]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):

    # Zero gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Set device options
    input_variable = input_variable.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)
    # Lengths for rnn packing should always be on the cpu
    lengths = lengths.to("cpu")

    # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0

    # Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    # Determine if we are using teacher forcing this iteration
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing: next input is current target
            decoder_input = target_variable[t].view(1, -1)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # No teacher forcing: next input is decoder's own current output
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    # Perform backpropatation
    loss.backward()

    # Clip gradients: gradients are modified in place
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # Adjust model weights
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

Training iterations + save our model to run inferences or continue training !

In [21]:
def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, loadFilename):

    # Load batches for each iteration
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]

    # Initializations
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1

    # Training loop
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        # Run a training iteration with batch
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

        # Print progress
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # Save checkpoint
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

## Gready decoding for generating sentences

Decoding method when training is not using teacher forcing. At each time step we choose the word from decoder_output with the highest softmax value.

In [22]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        # Forward input through encoder model
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        # Initialize decoder input with SOS_token
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
        # Initialize tensors to append decoded words to
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            # Obtain most likely word token and its softmax score
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # Return collections of word tokens and scores
        return all_tokens, all_scores

## Evaluation of model

In [23]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words


def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:
            # Get input sentence
            input_sentence = input('> ')
            # Check if it is quit case
            if input_sentence == 'q' or input_sentence == 'quit': break
            # Normalize sentence
            input_sentence = normalizeString(input_sentence)
            # Evaluate sentence
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            # Format and print response sentence
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Bot:', ' '.join(output_words))

        except KeyError:
            print("Error: Encountered unknown word.")

## Run model

possible to laod from checkpoint

In [25]:
# Configure models
model_name = 'cb_model'
attn_model = 'dot'
#attn_model = 'general'
#attn_model = 'concat'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64

# Set checkpoint to load from; set to None if starting from scratch
loadFilename = None
checkpoint_iter = 10000 # 4000
#loadFilename = os.path.join(save_dir, model_name, corpus_name,
#                            '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
#                            '{}_checkpoint.tar'.format(checkpoint_iter))
#print(loadFilename)

# Load model if a loadFilename is provided
if loadFilename:
    # If loading on same machine the model was trained on
    #checkpoint = torch.load(loadFilename)
    # If loading a model trained on GPU to CPU
    checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')
# Initialize word embeddings
embedding = nn.Embedding(voc.num_words, hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)
# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
# Use appropriate device
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


## Train !

In [26]:
# Configure training/optimization
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 10000 #4000
print_every = 1
save_every = 1000

# Ensure dropout layers are in train mode
encoder.train()
decoder.train()

# Initialize optimizers
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

# If you have cuda, configure cuda to call
for state in encoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

for state in decoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

# Run training iterations
print("Starting Training!")
trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, corpus_name, loadFilename)

Building optimizers ...
Starting Training!
Initializing ...
Training...
Iteration: 1; Percent complete: 0.0%; Average loss: 8.7289
Iteration: 2; Percent complete: 0.0%; Average loss: 8.6552
Iteration: 3; Percent complete: 0.0%; Average loss: 8.5558
Iteration: 4; Percent complete: 0.0%; Average loss: 8.3995
Iteration: 5; Percent complete: 0.1%; Average loss: 8.1517
Iteration: 6; Percent complete: 0.1%; Average loss: 7.7486
Iteration: 7; Percent complete: 0.1%; Average loss: 7.3293
Iteration: 8; Percent complete: 0.1%; Average loss: 7.2551
Iteration: 9; Percent complete: 0.1%; Average loss: 7.1287
Iteration: 10; Percent complete: 0.1%; Average loss: 6.9911
Iteration: 11; Percent complete: 0.1%; Average loss: 6.5238
Iteration: 12; Percent complete: 0.1%; Average loss: 6.3987
Iteration: 13; Percent complete: 0.1%; Average loss: 6.1331
Iteration: 14; Percent complete: 0.1%; Average loss: 5.8574
Iteration: 15; Percent complete: 0.1%; Average loss: 5.8495
Iteration: 16; Percent complete: 0.2%

Iteration: 138; Percent complete: 1.4%; Average loss: 4.2904
Iteration: 139; Percent complete: 1.4%; Average loss: 4.3727
Iteration: 140; Percent complete: 1.4%; Average loss: 4.6304
Iteration: 141; Percent complete: 1.4%; Average loss: 4.2700
Iteration: 142; Percent complete: 1.4%; Average loss: 4.1937
Iteration: 143; Percent complete: 1.4%; Average loss: 4.5301
Iteration: 144; Percent complete: 1.4%; Average loss: 4.5482
Iteration: 145; Percent complete: 1.5%; Average loss: 4.3067
Iteration: 146; Percent complete: 1.5%; Average loss: 4.4745
Iteration: 147; Percent complete: 1.5%; Average loss: 4.2464
Iteration: 148; Percent complete: 1.5%; Average loss: 4.5579
Iteration: 149; Percent complete: 1.5%; Average loss: 4.3710
Iteration: 150; Percent complete: 1.5%; Average loss: 4.5874
Iteration: 151; Percent complete: 1.5%; Average loss: 4.1929
Iteration: 152; Percent complete: 1.5%; Average loss: 4.2491
Iteration: 153; Percent complete: 1.5%; Average loss: 4.3406
Iteration: 154; Percent 

Iteration: 273; Percent complete: 2.7%; Average loss: 3.9472
Iteration: 274; Percent complete: 2.7%; Average loss: 4.0899
Iteration: 275; Percent complete: 2.8%; Average loss: 3.9513
Iteration: 276; Percent complete: 2.8%; Average loss: 4.0513
Iteration: 277; Percent complete: 2.8%; Average loss: 4.2249
Iteration: 278; Percent complete: 2.8%; Average loss: 3.7933
Iteration: 279; Percent complete: 2.8%; Average loss: 4.1180
Iteration: 280; Percent complete: 2.8%; Average loss: 4.0932
Iteration: 281; Percent complete: 2.8%; Average loss: 4.0167
Iteration: 282; Percent complete: 2.8%; Average loss: 4.1005
Iteration: 283; Percent complete: 2.8%; Average loss: 3.9898
Iteration: 284; Percent complete: 2.8%; Average loss: 3.9260
Iteration: 285; Percent complete: 2.9%; Average loss: 3.9588
Iteration: 286; Percent complete: 2.9%; Average loss: 3.7115
Iteration: 287; Percent complete: 2.9%; Average loss: 4.0338
Iteration: 288; Percent complete: 2.9%; Average loss: 4.0725
Iteration: 289; Percent 

Iteration: 408; Percent complete: 4.1%; Average loss: 3.6036
Iteration: 409; Percent complete: 4.1%; Average loss: 3.6759
Iteration: 410; Percent complete: 4.1%; Average loss: 3.9865
Iteration: 411; Percent complete: 4.1%; Average loss: 3.8071
Iteration: 412; Percent complete: 4.1%; Average loss: 3.8577
Iteration: 413; Percent complete: 4.1%; Average loss: 3.8897
Iteration: 414; Percent complete: 4.1%; Average loss: 3.6987
Iteration: 415; Percent complete: 4.2%; Average loss: 3.8259
Iteration: 416; Percent complete: 4.2%; Average loss: 3.9062
Iteration: 417; Percent complete: 4.2%; Average loss: 3.8808
Iteration: 418; Percent complete: 4.2%; Average loss: 4.0037
Iteration: 419; Percent complete: 4.2%; Average loss: 3.9889
Iteration: 420; Percent complete: 4.2%; Average loss: 3.6311
Iteration: 421; Percent complete: 4.2%; Average loss: 3.6098
Iteration: 422; Percent complete: 4.2%; Average loss: 3.6285
Iteration: 423; Percent complete: 4.2%; Average loss: 3.8442
Iteration: 424; Percent 

Iteration: 543; Percent complete: 5.4%; Average loss: 3.6995
Iteration: 544; Percent complete: 5.4%; Average loss: 3.5068
Iteration: 545; Percent complete: 5.5%; Average loss: 3.4398
Iteration: 546; Percent complete: 5.5%; Average loss: 3.7557
Iteration: 547; Percent complete: 5.5%; Average loss: 3.5011
Iteration: 548; Percent complete: 5.5%; Average loss: 3.6963
Iteration: 549; Percent complete: 5.5%; Average loss: 3.6516
Iteration: 550; Percent complete: 5.5%; Average loss: 3.6863
Iteration: 551; Percent complete: 5.5%; Average loss: 3.7272
Iteration: 552; Percent complete: 5.5%; Average loss: 3.7319
Iteration: 553; Percent complete: 5.5%; Average loss: 3.5531
Iteration: 554; Percent complete: 5.5%; Average loss: 3.5219
Iteration: 555; Percent complete: 5.5%; Average loss: 3.7882
Iteration: 556; Percent complete: 5.6%; Average loss: 3.7993
Iteration: 557; Percent complete: 5.6%; Average loss: 3.7256
Iteration: 558; Percent complete: 5.6%; Average loss: 3.8440
Iteration: 559; Percent 

...

Iteration: 9394; Percent complete: 93.9%; Average loss: 0.7213
Iteration: 9395; Percent complete: 94.0%; Average loss: 0.8395
Iteration: 9396; Percent complete: 94.0%; Average loss: 0.6734
Iteration: 9397; Percent complete: 94.0%; Average loss: 0.8079
Iteration: 9398; Percent complete: 94.0%; Average loss: 0.8148
Iteration: 9399; Percent complete: 94.0%; Average loss: 0.6536
Iteration: 9400; Percent complete: 94.0%; Average loss: 0.8095
Iteration: 9401; Percent complete: 94.0%; Average loss: 0.8091
Iteration: 9402; Percent complete: 94.0%; Average loss: 0.9187
Iteration: 9403; Percent complete: 94.0%; Average loss: 0.9378
Iteration: 9404; Percent complete: 94.0%; Average loss: 0.7133
Iteration: 9405; Percent complete: 94.0%; Average loss: 0.9928
Iteration: 9406; Percent complete: 94.1%; Average loss: 0.6129
Iteration: 9407; Percent complete: 94.1%; Average loss: 0.7941
Iteration: 9408; Percent complete: 94.1%; Average loss: 0.6978
Iteration: 9409; Percent complete: 94.1%; Average loss:

Iteration: 9527; Percent complete: 95.3%; Average loss: 0.7238
Iteration: 9528; Percent complete: 95.3%; Average loss: 0.7234
Iteration: 9529; Percent complete: 95.3%; Average loss: 0.6586
Iteration: 9530; Percent complete: 95.3%; Average loss: 0.6987
Iteration: 9531; Percent complete: 95.3%; Average loss: 0.6907
Iteration: 9532; Percent complete: 95.3%; Average loss: 0.7658
Iteration: 9533; Percent complete: 95.3%; Average loss: 0.7887
Iteration: 9534; Percent complete: 95.3%; Average loss: 0.6949
Iteration: 9535; Percent complete: 95.3%; Average loss: 0.7318
Iteration: 9536; Percent complete: 95.4%; Average loss: 0.6865
Iteration: 9537; Percent complete: 95.4%; Average loss: 0.7774
Iteration: 9538; Percent complete: 95.4%; Average loss: 0.5975
Iteration: 9539; Percent complete: 95.4%; Average loss: 0.7751
Iteration: 9540; Percent complete: 95.4%; Average loss: 0.6597
Iteration: 9541; Percent complete: 95.4%; Average loss: 0.7287
Iteration: 9542; Percent complete: 95.4%; Average loss:

Iteration: 9659; Percent complete: 96.6%; Average loss: 0.6203
Iteration: 9660; Percent complete: 96.6%; Average loss: 0.7768
Iteration: 9661; Percent complete: 96.6%; Average loss: 0.6101
Iteration: 9662; Percent complete: 96.6%; Average loss: 0.6430
Iteration: 9663; Percent complete: 96.6%; Average loss: 0.6555
Iteration: 9664; Percent complete: 96.6%; Average loss: 0.7568
Iteration: 9665; Percent complete: 96.7%; Average loss: 0.7020
Iteration: 9666; Percent complete: 96.7%; Average loss: 0.6481
Iteration: 9667; Percent complete: 96.7%; Average loss: 0.8281
Iteration: 9668; Percent complete: 96.7%; Average loss: 0.8477
Iteration: 9669; Percent complete: 96.7%; Average loss: 0.5789
Iteration: 9670; Percent complete: 96.7%; Average loss: 0.7581
Iteration: 9671; Percent complete: 96.7%; Average loss: 0.7244
Iteration: 9672; Percent complete: 96.7%; Average loss: 0.7012
Iteration: 9673; Percent complete: 96.7%; Average loss: 0.7848
Iteration: 9674; Percent complete: 96.7%; Average loss:

Iteration: 9792; Percent complete: 97.9%; Average loss: 0.6995
Iteration: 9793; Percent complete: 97.9%; Average loss: 0.7930
Iteration: 9794; Percent complete: 97.9%; Average loss: 0.7601
Iteration: 9795; Percent complete: 98.0%; Average loss: 0.8120
Iteration: 9796; Percent complete: 98.0%; Average loss: 0.6549
Iteration: 9797; Percent complete: 98.0%; Average loss: 0.6419
Iteration: 9798; Percent complete: 98.0%; Average loss: 0.6740
Iteration: 9799; Percent complete: 98.0%; Average loss: 0.6716
Iteration: 9800; Percent complete: 98.0%; Average loss: 0.7913
Iteration: 9801; Percent complete: 98.0%; Average loss: 0.7229
Iteration: 9802; Percent complete: 98.0%; Average loss: 0.7746
Iteration: 9803; Percent complete: 98.0%; Average loss: 0.7891
Iteration: 9804; Percent complete: 98.0%; Average loss: 0.6182
Iteration: 9805; Percent complete: 98.0%; Average loss: 0.7876
Iteration: 9806; Percent complete: 98.1%; Average loss: 0.8382
Iteration: 9807; Percent complete: 98.1%; Average loss:

Iteration: 9923; Percent complete: 99.2%; Average loss: 0.6682
Iteration: 9924; Percent complete: 99.2%; Average loss: 0.6235
Iteration: 9925; Percent complete: 99.2%; Average loss: 0.7870
Iteration: 9926; Percent complete: 99.3%; Average loss: 0.7003
Iteration: 9927; Percent complete: 99.3%; Average loss: 0.7120
Iteration: 9928; Percent complete: 99.3%; Average loss: 0.5562
Iteration: 9929; Percent complete: 99.3%; Average loss: 0.6617
Iteration: 9930; Percent complete: 99.3%; Average loss: 0.6167
Iteration: 9931; Percent complete: 99.3%; Average loss: 0.6314
Iteration: 9932; Percent complete: 99.3%; Average loss: 0.7648
Iteration: 9933; Percent complete: 99.3%; Average loss: 0.6627
Iteration: 9934; Percent complete: 99.3%; Average loss: 0.7917
Iteration: 9935; Percent complete: 99.4%; Average loss: 0.7065
Iteration: 9936; Percent complete: 99.4%; Average loss: 0.8010
Iteration: 9937; Percent complete: 99.4%; Average loss: 0.6216
Iteration: 9938; Percent complete: 99.4%; Average loss:

## RUN AN PLAY

In [28]:
# Set dropout layers to eval mode
encoder.eval()
decoder.eval()

# Initialize search module
searcher = GreedySearchDecoder(encoder, decoder)

# Begin chatting
#evaluateInput(encoder, decoder, searcher, voc)

## Reinforcement learning

### Forward and backward models 

In [29]:
#forward_encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
#forward_decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
#forward_encoder = forward_encoder.to(device)
#forward_decoder = forward_decoder.to(device)

In [30]:
forward_encoder = encoder
forward_decoder = decoder
forward_encoder = forward_encoder.to(device)
forward_decoder = forward_decoder.to(device)

In [31]:
backward_encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
backward_decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
backward_encoder = backward_encoder.to(device)
backward_decoder = backward_decoder.to(device)

### Utility functions

In [32]:
def convertResponse(response):
    size1 = len(response)
    size2 = batch_size
    npRes = np.zeros((size1, size2), dtype=np.int64)
    npLengths = np.zeros(size2, dtype=np.int64)
    for i in range(size1):
        prov = response[i].cpu().numpy()
        for j in range(prov.size):
            npLengths[j] = npLengths[j] + 1
            if prov.size > 1:
                npRes[i][j] = prov[j]
            else:
                npRes[i][j] = prov 
    res = torch.from_numpy(npRes)
    lengths = torch.from_numpy(npLengths)
    return res, lengths

In [33]:
def convertTarget(target):
    size1 = len(target)
    size2 = batch_size
    npRes = np.zeros((size1, size2), dtype=np.int64)
    mask = np.zeros((size1, size2), dtype=np.bool_)
    npLengths = np.zeros(size2, dtype=np.int64)
    for i in range(size1):
        prov = target[i].cpu().numpy()
        for j in range(prov.size):
            npLengths[j] = npLengths[j] + 1
            if prov.size > 1:
                npRes[i][j] = prov[j]
            else:
                npRes[i][j] = prov 
                
            if npRes[i][j] > 0:
                mask[i][j] = True
            else:
                mask[i][j] = False
            
    res = torch.from_numpy(npRes)
    lengths = torch.from_numpy(npLengths)
    mask= torch.from_numpy(mask)
    max_target_len = torch.max(lengths) #.detach().numpy()
    return res, mask, max_target_len

In [34]:
def transformTensorToSameShapeAs(tensor, shape):
    size1, size2 = shape
    npNewT = np.zeros((size1, size2), dtype=np.int64)
    npNewMask = np.zeros((size1, size2), dtype=np.bool_)
    tensorSize1, tensorSize2 = tensor.size()
    for i in range(tensorSize1):
        for j in range(tensorSize2):
            npNewT[i][j] = tensor[i][j]
            npNewMask[i][j]= True
    return torch.from_numpy(npNewT), torch.from_numpy(npNewMask)

### Training step for a single iteration 

In [35]:
def RL(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, batch_size, teacher_forcing_ratio):
    # Set device options
    input_variable = input_variable.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)
    # Lengths for rnn packing should always be on the cpu
    lengths = lengths.to("cpu")
    
    #Initialize variables
    loss=0
    #print_losses = []
    response=[]
    
    # Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)
    
    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)
    
    
    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:decoder.n_layers]

     # Determine if we are using teacher forcing this iteration
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    
     # Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing: next input is current target
            decoder_input = target_variable[t].view(1, -1)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            #print_losses.append(mask_loss.item() * nTotal)
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # No teacher forcing: next input is decoder's own current output
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            #print_losses.append(mask_loss.item() * nTotal)
            
            #ni or decoder_output
            response.append(topi)
            
    return loss, max_target_len, response

Let's define the rewards method for each action.

It is composed of 3 different types of rewards :
- `ease of answering` which is basically saying that a turn generated by a machineshould be easy to respond to. 
- `information flow` : We  want  the  agent  to  contribute new information at each turn to keep the dialogue moving and avoid repetitive sequences. 
- `semantic coherence` measures the adequacy of responses to avoid situations in which the generated replies are highly rewarded but are ungrammatical or not coherent. 

In [36]:
def easeOfAnswering(input_variable, lengths, dull_responses, mask, max_target_len, encoder, decoder, batch_size, teacher_forcing_ratio):
    NS=len(dull_responses)
    r1=0
    for d in dull_responses:
        d, mask, max_target_len = outputVar(d, voc)
        newD, newMask = transformTensorToSameShapeAs(d, input_variable.size())
        #tar, mask, max_target_len = convertTarget(d)
        forward_loss, forward_len, _ = RL(input_variable, lengths, newD, newMask, max_target_len, encoder, decoder, batch_size, teacher_forcing_ratio)
        # log (1/P(a|s)) = CE  --> log(P(a | s)) = - CE
        if forward_len > 0:
            r1 -= forward_loss / forward_len
    if len(dull_responses) > 0:
        r1 = r1 / NS
    return r1

In [37]:
def informationFlow(responses):
    r2=0
    if(len(responses) > 2):
        #2 representations obtained from the encoder for two consecutive turns pi and pi+1
        h_pi = responses[-3]
        h_pi1 = responses[-1]
        # length of the two vector might not match
        min_length = min(len(h_pi), len(h_pi+1))
        h_pi = h_pi[:min_length]
        h_pi1 = h_pi1[:min_length]
        #cosine similarity 
        #cos_sim = 1 - distance.cosine(h_pi, h_pi1)
        cos_sim = 1 - distance.cdist(h_pi.cpu().numpy(), h_pi1.cpu().numpy(), 'cosine')
        #Handle negative cos_sim
        if np.any(cos_sim <= 0):
            r2 = - cos_sim
        else:
            r2 = - np.log(cos_sim)
        r2 = np.mean(r2)
    return r2

In [38]:
def semanticCoherence(input_variable, lengths, target_variable, mask, max_target_len, forward_encoder, forward_decoder, backward_encoder, backward_decoder, batch_size, teacher_forcing_ratio):
    #print("IN R3:")
    #print("Input_variable :", input_variable.shape)
    #print("Lengths :", lengths.shape)
    #print("Target_variable :", target_variable.shape)
    #print("Mask :", mask.shape)
    #print("Max_Target_Len :", max_target_len)
    r3 = 0
    forward_loss, forward_len, _ = RL(input_variable, lengths, target_variable, mask, max_target_len, forward_encoder, forward_decoder, batch_size, teacher_forcing_ratio)
    ep_input, lengths_trans = convertResponse(target_variable)
    #print("ep_input :", ep_input.shape)
    #print("Lengths transformed :", lengths_trans.shape)
    ep_target, mask_trans, max_target_len_trans = convertTarget(input_variable)
    #print("ep_target :", ep_target.shape)
    #print("mask transformed :", mask_trans.shape)
    #print("max_target_len_trans :", max_target_len_trans)
    backward_loss, backward_len, _ = RL(ep_input, lengths_trans, ep_target, mask_trans, max_target_len_trans, backward_encoder, backward_decoder, batch_size, teacher_forcing_ratio)
    if forward_len > 0:
        r3 += forward_loss / forward_len
    if backward_len > 0:
        r3+= backward_loss / backward_len
    return r3

In [39]:
l1=0.25
l2=0.25
l3=0.5

In [40]:
def calculate_rewards(input_var, lengths, target_var, mask, max_target_len, forward_encoder, forward_decoder, backward_encoder, backward_decoder, batch_size, teacher_forcing_ratio):
    #rewards per episode
    ep_rewards = []
    #indice of current episode
    ep_num = 1
    #list of responses
    responses = []
    #input of current episode
    ep_input = input_var
    #target of current episode
    ep_target = target_var
    
    #ep_num bounded -> to redefine (MEDIUM POST)
    while (ep_num <= 10):
        
        print(ep_num)
        #generate current response with the forward model
        _, _, curr_response = RL(ep_input, lengths, ep_target, mask, max_target_len, forward_encoder, forward_decoder, batch_size, teacher_forcing_ratio)
        
        #Break if :
        # 1 -> dull response
        # 2 -> response is less than MIN_LENGTH
        # 3 -> repetition ie curr_response in responses
        if(len(curr_response) < MIN_COUNT):# or (curr_response in dull_responses) or (curr_response in responses)):
            break
            
            
        #We can add the response to responses list
        #curr_response = torch.LongTensor(curr_response).view(-1, 1)
        #transform curr_response size
        #target = torch.zeros(960, 1)
        #target[:15, :] = curr_response
        #curr_response = target
        #print(curr_response.size())
        #curr_response = torch.reshape(curr_response, (15, 64))
        #print(curr_response.size())
        #curr_response = curr_response.to(device)
        #responses.append(curr_response) 
        
        
        #Ease of answering
        r1 = easeOfAnswering(ep_input, lengths, dull_responses, mask, max_target_len, forward_encoder, forward_decoder, batch_size, teacher_forcing_ratio)
        
        #Information flow
        r2 = informationFlow(responses)
        
        #Semantic coherence
        r3 = semanticCoherence(ep_input, lengths, target_var, mask, max_target_len, forward_encoder, forward_decoder, backward_encoder, backward_decoder, batch_size, teacher_forcing_ratio)
        
        #Final reward as a weighted sum of rewards
        r = l1*r1 + l2*r2 + l3*r3
        
        #Add the current reward to the list
        ep_rewards.append(r.detach().cpu().numpy())
        
        #We can add the response to responses list
        curr_response, lengths = convertResponse(curr_response)
        curr_response = curr_response.to(device)
        responses.append(curr_response)
        
        #Next input is the current response
        ep_input = curr_response
        #Next target -> dummy
        ep_target = torch.zeros(MAX_LENGTH,batch_size,dtype=torch.int64)
        #ep_target = torch.LongTensor(torch.LongTensor([0] * MAX_LENGTH)).view(-1, 1)
        ep_target = ep_target.to(device)
        
        #Turn off the teacher forcing  after first iteration -> dummy target
        teacher_forcing_ratio = 0
        ep_num +=1
        
    #Take the mean of the episodic rewards
    return np.mean(ep_rewards) if len(ep_rewards) > 0 else 0 

Dull responses 

In [41]:
dull_responses = ["i do not know what you are talking about.", "i do not know.", 
 "you do not know.", "you know what i mean.", "i know what you mean.", 
 "you know what i am saying.", "you do not know anything."]
#dull_responses = []

### Training RL loop

In [42]:
def trainingRLLoop(model_name, voc, pairs, batch_size, forward_encoder, forward_encoder_optimizer, forward_decoder, forward_decoder_optimizer, backward_encoder, backward_encoder_optimizer, backward_decoder, backward_decoder_optimizer,teacher_forcing_ratio, dull_responses, n_iteration, print_every, save_every, save_dir):
    
    # Load batches for each iteration
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]
    
    # Initializations
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    
    
    #Training loop
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        print("Iteration", iteration)
        training_batch = training_batches[iteration - 1]
        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        ##MODIFS HERE
        # Zero gradients the optimizer
        forward_encoder_optimizer.zero_grad()
        forward_decoder_optimizer.zero_grad()
        
        backward_encoder_optimizer.zero_grad()
        backward_decoder_optimizer.zero_grad()
        
        #Forward
        forward_loss, forward_len, _ = RL(input_variable, lengths, target_variable, mask, max_target_len, forward_encoder, forward_decoder, batch_size, teacher_forcing_ratio)
        
        #Calculate reward
        reward = calculate_rewards(input_variable, lengths, target_variable, mask, max_target_len, forward_encoder, forward_decoder, backward_encoder, backward_decoder, batch_size, teacher_forcing_ratio)
        
        #Update forward seq2seq with loss scaled by reward
        loss = forward_loss * reward
        
        loss.backward()
        forward_encoder_optimizer.step()
        forward_decoder_optimizer.step()

        # Run a training iteration with batch
        print_loss += loss / forward_len
        
        # Print progress
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0
            
        #SAVE CHECKPOINT TO DO
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name)#, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

## RUN 

In [ ]:
#Configure RL model

model_name='RL_model_seq'
n_iteration = 10000
print_every=100
save_every=500
learning_rate = 0.0001
decoder_learning_ratio = 5.0
teacher_forcing_ratio = 0.5

# Ensure dropout layers are in train mode
forward_encoder.train()
forward_decoder.train()

backward_encoder.train()
backward_decoder.train()

# Initialize optimizers
print('Building optimizers ...')
forward_encoder_optimizer = optim.Adam(forward_encoder.parameters(), lr=learning_rate)
forward_decoder_optimizer = optim.Adam(forward_decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
backward_encoder_optimizer = optim.Adam(backward_encoder.parameters(), lr=learning_rate)
backward_decoder_optimizer = optim.Adam(backward_decoder.parameters(), lr=learning_rate * decoder_learning_ratio)

# If you have cuda, configure cuda to call
for state in forward_encoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

for state in forward_decoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()
            
for state in backward_encoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

for state in backward_decoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()
            
# Run training iterations
print("Starting Training!")
trainingRLLoop(model_name, voc, pairs, batch_size, forward_encoder, forward_encoder_optimizer, forward_decoder, forward_decoder_optimizer, backward_encoder, backward_encoder_optimizer, backward_decoder, backward_decoder_optimizer, teacher_forcing_ratio, dull_responses, n_iteration, print_every, save_every, save_dir)

Building optimizers ...
Starting Training!
Initializing ...
Training...
Iteration 1
1
Iteration 2
1
2
3
4
5
6
7
8
9
10
Iteration 3
1
2
3
4
5
6
7
8
9
10
Iteration 4
1
Iteration 5
1
Iteration 6
1
Iteration 7
1
Iteration 8
1
Iteration 9
1
Iteration 10
1
Iteration 11
1
2
3
4
5
6
7
8
9
10
Iteration 12
1
Iteration 13
1
Iteration 14
1
2
3
4
5
6
7
8
9
10
Iteration 15
1
Iteration 16
1
2
3
4
5
6
7
8
9
10
Iteration 17
1
2
3
4
5
6
7
8
9
10
Iteration 18
1
Iteration 19
1
Iteration 20
1
2
3
4
5
6
7
8
9
10
Iteration 21
1
Iteration 22
1
Iteration 23
1
Iteration 24
1
Iteration 25
1
2
3
4
5
6
7
8
9
10
Iteration 26
1
2
3
4
5
6
7
8
9
10
Iteration 27
1
Iteration 28
1
2
3
4
5
6
7
8
9
10
Iteration 29
1
Iteration 30
1
2
3
4
5
6
7
8
9
10
Iteration 31
1
Iteration 32
1
Iteration 33
1
2
3
4
5
6
7
8
9
10
Iteration 34
1
2
3
4
5
6
7
8
9
10
Iteration 35
1
2
3
4
5
6
7
8
9
10
Iteration 36
1
2
3
4
5
6
7
8
9
10
Iteration 37
1
Iteration 38
1
Iteration 39
1
Iteration 40
1
2
3
4
5
6
7
8
9
10
Iteration 41
1
Iteration 42
1
2
3

4
5
6
7
8
9
10
Iteration 322
1
Iteration 323
1
2
3
4
5
6
7
8
9
10
Iteration 324
1
Iteration 325
1
Iteration 326
1
Iteration 327
1
Iteration 328
1
2
3
4
5
6
7
8
9
10
Iteration 329
1
Iteration 330
1
Iteration 331
1
2
3
4
5
6
7
8
9
10
Iteration 332
1
2
3
4
5
6
7
8
9
10
Iteration 333
1
2
3
4
5
6
7
8
9
10
Iteration 334
1
2
3
4
5
6
7
8
9
10
Iteration 335
1
Iteration 336
1
Iteration 337
1
2
3
4
5
6
7
8
9
10
Iteration 338
1
2
3
4
5
6
7
8
9
10
Iteration 339
1
2
3
4
5
6
7
8
9
10
Iteration 340
1
Iteration 341
1
Iteration 342
1
Iteration 343
1
2
3
4
5
6
7
8
9
10
Iteration 344
1
Iteration 345
1
2
3
4
5
6
7
8
9
10
Iteration 346
1
Iteration 347
1
Iteration 348
1
Iteration 349
1
Iteration 350
1
2
3
4
5
6
7
8
9
10
Iteration 351
1
2
3
4
5
6
7
8
9
10
Iteration 352
1
Iteration 353
1
2
3
4
5
6
7
8
9
10
Iteration 354
1
Iteration 355
1
2
3
4
5
6
7
8
9
10
Iteration 356
1
Iteration 357
1
2
3
4
5
6
7
8
9
10
Iteration 358
1
2
3
4
5
6
7
8
9
10
Iteration 359
1
Iteration 360
1
2
3
4
5
6
7
8
9
10
Iteration 361
1
Ite

4
5
6
7
8
9
10
Iteration 635
1
Iteration 636
1
Iteration 637
1
Iteration 638
1
Iteration 639
1
2
3
4
5
6
7
8
9
10
Iteration 640
1
Iteration 641
1
Iteration 642
1
Iteration 643
1
Iteration 644
1
2
3
4
5
6
7
8
9
10
Iteration 645
1
2
3
4
5
6
7
8
9
10
Iteration 646
1
2
3
4
5
6
7
8
9
10
Iteration 647
1
Iteration 648
1
2
3
4
5
6
7
8
9
10
Iteration 649
1
Iteration 650
1
2
3
4
5
6
7
8
9
10
Iteration 651
1
2
3
4
5
6
7
8
9
10
Iteration 652
1
2
3
4
5
6
7
8
9
10
Iteration 653
1
2
3
4
5
6
7
8
9
10
Iteration 654
1
Iteration 655
1
2
3
4
5
6
7
8
9
10
Iteration 656
1
Iteration 657
1
2
3
4
5
6
7
8
9
10
Iteration 658
1
2
3
4
5
6
7
8
9
10
Iteration 659
1
Iteration 660
1
2
3
4
5
6
7
8
9
10
Iteration 661
1
Iteration 662
1
Iteration 663
1
Iteration 664
1
Iteration 665
1
2
3
4
5
6
7
8
9
10
Iteration 666
1
2
3
4
5
6
7
8
9
10
Iteration 667
1
Iteration 668
1
2
3
4
5
6
7
8
9
10
Iteration 669
1
2
3
4
5
6
7
8
9
10
Iteration 670
1
Iteration 671
1
Iteration 672
1
Iteration 673
1
2
3
4
5
6
7
8
9
10
Iteration 674
1
2
3

1
Iteration 951
1
2
3
4
5
6
7
8
9
10
Iteration 952
1
2
3
4
5
6
7
8
9
10
Iteration 953
1
2
3
4
5
6
7
8
9
10
Iteration 954
1
Iteration 955
1
Iteration 956
1
2
3
4
5
6
7
8
9
10
Iteration 957
1
Iteration 958
1
Iteration 959
1
2
3
4
5
6
7
8
9
10
Iteration 960
1
2
3
4
5
6
7
8
9
10
Iteration 961
1
2
3
4
5
6
7
8
9
10
Iteration 962
1
Iteration 963
1
Iteration 964
1
Iteration 965
1
2
3
4
5
6
7
8
9
10
Iteration 966
1
Iteration 967
1
Iteration 968
1
2
3
4
5
6
7
8
9
10
Iteration 969
1
Iteration 970
1
Iteration 971
1
Iteration 972
1
Iteration 973
1
2
3
4
5
6
7
8
9
10
Iteration 974
1
2
3
4
5
6
7
8
9
10
Iteration 975
1
2
3
4
5
6
7
8
9
10
Iteration 976
1
Iteration 977
1
Iteration 978
1
Iteration 979
1
Iteration 980
1
Iteration 981
1
Iteration 982
1
2
3
4
5
6
7
8
9
10
Iteration 983
1
2
3
4
5
6
7
8
9
10
Iteration 984
1
2
3
4
5
6
7
8
9
10
Iteration 985
1
2
3
4
5
6
7
8
9
10
Iteration 986
1
2
3
4
5
6
7
8
9
10
Iteration 987
1
2
3
4
5
6
7
8
9
10
Iteration 988
1
Iteration 989
1
2
3
4
5
6
7
8
9
10
Iteration 990

Iteration 1248
1
2
3
4
5
6
7
8
9
10
Iteration 1249
1
2
3
4
5
6
7
8
9
10
Iteration 1250
1
2
3
4
5
6
7
8
9
10
Iteration 1251
1
Iteration 1252
1
2
3
4
5
6
7
8
9
10
Iteration 1253
1
2
3
4
5
6
7
8
9
10
Iteration 1254
1
Iteration 1255
1
Iteration 1256
1
Iteration 1257
1
2
3
4
5
6
7
8
9
10
Iteration 1258
1
Iteration 1259
1
2
3
4
5
6
7
8
9
10
Iteration 1260
1
Iteration 1261
1
Iteration 1262
1
2
3
4
5
6
7
8
9
10
Iteration 1263
1
2
3
4
5
6
7
8
9
10
Iteration 1264
1
Iteration 1265
1
Iteration 1266
1
Iteration 1267
1
Iteration 1268
1
Iteration 1269
1
Iteration 1270
1
2
3
4
5
6
7
8
9
10
Iteration 1271
1
Iteration 1272
1
2
3
4
5
6
7
8
9
10
Iteration 1273
1
Iteration 1274
1
Iteration 1275
1
Iteration 1276
1
Iteration 1277
1
2
3
4
5
6
7
8
9
10
Iteration 1278
1
2
3
4
5
6
7
8
9
10
Iteration 1279
1
2
3
4
5
6
7
8
9
10
Iteration 1280
1
Iteration 1281
1
2
3
4
5
6
7
8
9
10
Iteration 1282
1
2
3
4
5
6
7
8
9
10
Iteration 1283
1
Iteration 1284
1
Iteration 1285
1
Iteration 1286
1
Iteration 1287
1
2
3
4
5
6
7
8
9


Iteration 1563
1
2
3
4
5
6
7
8
9
10
Iteration 1564
1
Iteration 1565
1
2
3
4
5
6
7
8
9
10
Iteration 1566
1
2
3
4
5
6
7
8
9
10
Iteration 1567
1
Iteration 1568
1
2
3
4
5
6
7
8
9
10
Iteration 1569
1
Iteration 1570
1
Iteration 1571
1
Iteration 1572
1
Iteration 1573
1
Iteration 1574
1
Iteration 1575
1
2
3
4
5
6
7
8
9
10
Iteration 1576
1
Iteration 1577
1
2
3
4
5
6
7
8
9
10
Iteration 1578
1
2
3
4
5
6
7
8
9
10
Iteration 1579
1
2
3
4
5
6
7
8
9
10
Iteration 1580
1
Iteration 1581
1
Iteration 1582
1
Iteration 1583
1
Iteration 1584
1
Iteration 1585
1
2
3
4
5
6
7
8
9
10
Iteration 1586
1
2
3
4
5
6
7
8
9
10
Iteration 1587
1
Iteration 1588
1
Iteration 1589
1
Iteration 1590
1
Iteration 1591
1
Iteration 1592
1
2
3
4
5
6
7
8
9
10
Iteration 1593
1
Iteration 1594
1
2
3
4
5
6
7
8
9
10
Iteration 1595
1
2
3
4
5
6
7
8
9
10
Iteration 1596
1
2
3
4
5
6
7
8
9
10
Iteration 1597
1
Iteration 1598
1
2
3
4
5
6
7
8
9
10
Iteration 1599
1
Iteration 1600
1
Iteration: 1600; Percent complete: 16.0%; Average loss: 4.3134
Iterat

Iteration 1862
1
Iteration 1863
1
2
3
4
5
6
7
8
9
10
Iteration 1864
1
Iteration 1865
1
2
3
4
5
6
7
8
9
10
Iteration 1866
1
2
3
4
5
6
7
8
9
10
Iteration 1867
1
Iteration 1868
1
Iteration 1869
1
2
3
4
5
6
7
8
9
10
Iteration 1870
1
Iteration 1871
1
Iteration 1872
1
2
3
4
5
6
7
8
9
10
Iteration 1873
1
2
3
4
5
6
7
8
9
10
Iteration 1874
1
Iteration 1875
1
Iteration 1876
1
2
3
4
5
6
7
8
9
10
Iteration 1877
1
2
3
4
5
6
7
8
9
10
Iteration 1878
1
2
3
4
5
6
7
8
9
10
Iteration 1879
1
2
3
4
5
6
7
8
9
10
Iteration 1880
1
2
3
4
5
6
7
8
9
10
Iteration 1881
1
2
3
4
5
6
7
8
9
10
Iteration 1882
1
Iteration 1883
1
Iteration 1884
1
2
3
4
5
6
7
8
9
10
Iteration 1885
1
Iteration 1886
1
2
3
4
5
6
7
8
9
10
Iteration 1887
1
2
3
4
5
6
7
8
9
10
Iteration 1888
1
Iteration 1889
1
Iteration 1890
1
2
3
4
5
6
7
8
9
10
Iteration 1891
1
2
3
4
5
6
7
8
9
10
Iteration 1892
1
Iteration 1893
1
2
3
4
5
6
7
8
9
10
Iteration 1894
1
2
3
4
5
6
7
8
9
10
Iteration 1895
1
Iteration 1896
1
2
3
4
5
6
7
8
9
10
Iteration 1897
1
2
3
4
5


5
6
7
8
9
10
Iteration 2166
1
2
3
4
5
6
7
8
9
10
Iteration 2167
1
Iteration 2168
1
Iteration 2169
1
2
3
4
5
6
7
8
9
10
Iteration 2170
1
Iteration 2171
1
2
3
4
5
6
7
8
9
10
Iteration 2172
1
Iteration 2173
1
Iteration 2174
1
2
3
4
5
6
7
8
9
10
Iteration 2175
1
2
3
4
5
6
7
8
9
10
Iteration 2176
1
Iteration 2177
1
2
3
4
5
6
7
8
9
10
Iteration 2178
1
2
3
4
5
6
7
8
9
10
Iteration 2179
1
2
3
4
5
6
7
8
9
10
Iteration 2180
1
2
3
4
5
6
7
8
9
10
Iteration 2181
1
2
3
4
5
6
7
8
9
10
Iteration 2182
1
Iteration 2183
1
Iteration 2184
1
Iteration 2185
1
2
3
4
5
6
7
8
9
10
Iteration 2186
1
Iteration 2187
1
Iteration 2188
1
Iteration 2189
1
Iteration 2190
1
2
3
4
5
6
7
8
9
10
Iteration 2191
1
Iteration 2192
1
Iteration 2193
1
Iteration 2194
1
2
3
4
5
6
7
8
9
10
Iteration 2195
1
Iteration 2196
1
Iteration 2197
1
2
3
4
5
6
7
8
9
10
Iteration 2198
1
2
3
4
5
6
7
8
9
10
Iteration 2199
1
Iteration 2200
1
2
3
4
5
6
7
8
9
10
Iteration: 2200; Percent complete: 22.0%; Average loss: 4.2678
Iteration 2201
1
2
3
4
5


2
3
4
5
6
7
8
9
10
Iteration 2473
1
2
3
4
5
6
7
8
9
10
Iteration 2474
1
Iteration 2475
1
2
3
4
5
6
7
8
9
10
Iteration 2476
1
Iteration 2477
1
Iteration 2478
1
Iteration 2479
1
Iteration 2480
1
2
3
4
5
6
7
8
9
10
Iteration 2481
1
Iteration 2482
1
2
3
4
5
6
7
8
9
10
Iteration 2483
1
2
3
4
5
6
7
8
9
10
Iteration 2484
1
Iteration 2485
1
Iteration 2486
1
2
3
4
5
6
7
8
9
10
Iteration 2487
1
Iteration 2488
1
Iteration 2489
1
2
3
4
5
6
7
8
9
10
Iteration 2490
1
Iteration 2491
1
2
3
4
5
6
7
8
9
10
Iteration 2492
1
Iteration 2493
1
2
3
4
5
6
7
8
9
10
Iteration 2494
1
Iteration 2495
1
2
3
4
5
6
7
8
9
10
Iteration 2496
1
2
3
4
5
6
7
8
9
10
Iteration 2497
1
2
3
4
5
6
7
8
9
10
Iteration 2498
1
Iteration 2499
1
Iteration 2500
1
2
3
4
5
6
7
8
9
10
Iteration: 2500; Percent complete: 25.0%; Average loss: 4.5662
Iteration 2501
1
Iteration 2502
1
2
3
4
5
6
7
8
9
10
Iteration 2503
1
2
3
4
5
6
7
8
9
10
Iteration 2504
1
2
3
4
5
6
7
8
9
10
Iteration 2505
1
2
3
4
5
6
7
8
9
10
Iteration 2506
1
2
3
4
5
6
7
8
9
1

In [ ]:
# Set dropout layers to eval mode
forward_encoder.eval()
forward_decoder.eval()

# Initialize search module
searcher = GreedySearchDecoder(forward_encoder, forward_decoder)

# Begin chatting
evaluateInput(forward_encoder, forward_decoder, searcher, voc)